# 교차검증: cross_val_score()

- 데이터 편중을 막기 위해서 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행하는 것


1. K폴드 교차검증
    - K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법
2. Stratified K폴드 교차검증
    - 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K폴드 방식
    - 불균형한 분포도를 가진 레이블 데이터: 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것
        - 예) 대출 사기 데이터의 레이블은 주로 0(정상 대출)은 매우 많고, 1(대출 사기)는 매우 적음
    - 원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 유사하게 학습과 검증 데이터 세트를 분배함
    - 분류에서는 주로 Stratified K폴드 교차검증을 사용하고 회귀에서는 사용하지 않음
    
```python
cross_val_score(estimator, X, y = None, scoring = None, cv = None, n_jobs = 1, verbose = 0, fit_params = None, pre_dispatch = '2*n_jobs')
```

- 주요 파라미터
    - estimator: 사이킷런의 분류 알고리즘 클래스인 Classifier 또는 회귀 알고리즘 클래스인 Regressor를 의미
    - X: feature 데이터 세트
    - y: 레이블 데이터 세트
    - scoring: 예측 성능 평가 지표
    - cv: 교차 검증 폴드 수
- cross_val_score는 estimator에 classifier가 입력되면 Stratified K폴드 방식으로, 회귀인 경우는 K폴드 방식으로 분할함

In [1]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 156)

data = iris_data.data
label = iris_data.target

# 교차 검증

scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv = 3)
print('교차 검증별 정확도: ', np.round(scores, 4))
print('평균 검증 정확도: ', np.round(np.mean(scores), 4))

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도:  0.9667


- cross_val_score()는 단 하나의 평가 지표만 가능하지만
- cross_validate()는 여러 개의 평가 지표를 반환할 수 있음. 또한, 학습 데이터에 대한 성능 평가 지표와 수행 시간도 같이 제공함

# GridSearchCV: 교차 검증과 최적 하이퍼 파라미터 튜닝을 수행

- 교차 검증을 기반으로 하이퍼 파라미터의 최적 값을 찾음
- GridSearchCV의 주요 파라미터
    - estimator: classifier, regressor, pipeline이 사용될 수 있음
    - param_grid: key + 리스트 값을 가지는 딕셔너리. estimator의 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정
    - scoring: 예측 성능을 측정할 평가 방법 지정
    - cv: 교차 검증을 위해 분할되는 학습/테스트 세트의 개수 지정
    - refit: 디폴트가 True. True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킴

In [2]:
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로딩 및 학습 데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size = 0.2, random_state = 121)
dtree = DecisionTreeClassifier()

# 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}

# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정(cv = 3)
grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True)

# 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000,0.000000,0.000000,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5
1,0.000000,0.000000,0.000334,0.000472,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5
2,0.000666,0.000471,0.000000,0.000000,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3
3,0.000000,0.000000,0.000334,0.000472,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3
4,0.000335,0.000474,0.000000,0.000000,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1
5,0.000331,0.000468,0.000000,0.000000,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1


In [3]:
# 주요 컬럼만 발췌

scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


- params: 수행할 때마다 적용된 개별 하이퍼 파라미터 값
- rank_test_score: 하이퍼 파라미터별로 성능이 좋은 score 순위. 1이 가장 뛰어난 순위이며 이때의 파라미터가 최적의 하이퍼 파라미터
- mean_test_score: 개별 하이퍼 파라미터별로 CV의 폴딩 테스트 세트에 대해 총 수행한 평가 평균값
- split0_test_score, split1_test_score, split2_test_score: CV가 3인 경우, 즉 3개의 폴딩 세트에서 각각 테스트한 성능 수치
    - mean_test_score는 이 3개의 성능 수치를 평균한 것

In [4]:
print('GridSearchCV 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750


In [5]:
# GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimatr_는 이미 최적 학습이 됐으므로 별도 학습 필요 없음
# 따라서 이를 이용해 테스트 데이터 세트에 대해 예측하고 성능 평가
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도: 0.9667
